In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import collections

from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model

from sklearn.metrics import accuracy_score

In [ ]:
class Classifier:
    def __init__(self,numClasses, input_shape, batch_size, epochs):
        # with strategy.scope():
            self.numClasses = numClasses
            self.input_shape = input_shape
            self.batch_size = batch_size
            self.epochs = epochs
            

            self.model = Sequential()

            self.model.add(Conv2D(64,kernel_size=2,
                            activation='relu',
                            padding= 'same',
                            input_shape=input_shape, data_format='channels_last'))

            self.model.add(Conv2D(128, (3, 3), activation='relu'))
            self.model.add(MaxPooling2D(pool_size=(2, 2)))
            
            self.model.add(Conv2D(128, (3, 3), activation='relu'))
            self.model.add(MaxPooling2D(pool_size=(2, 2)))

            self.model.add(Dropout(.5))
            
            self.model.add(Flatten())

            self.model.add(Dense(128, activation='relu'))
            self.model.add(Dropout(.5))
            
            self.model.add(Dense(self.numClasses, activation='softmax'))



            sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
            adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

            
            self.model.compile(loss='categorical_crossentropy',
                          optimizer= adam,
                          metrics=['accuracy'])
            
            self.model.summary()


    def Train(self, Xtrain, ytrain, Xtest, ytest, model_filepath):
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
        mc = ModelCheckpoint(model_filepath, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
        self.model.fit(Xtrain, ytrain,
                    validation_data= (Xtest, ytest), 
                    epochs=self.epochs,
                    shuffle=True,
                    verbose = 1, 
                    callbacks= [es, mc])
          
    def Test(self, Xtest, ytest): 
        if('A' in ytest):
           ytest = keras.utils.to_categorical(charToInt(ytest));
       
        predictions = self.model.predict(Xtest)
        y_pred = np.argmax(predictions, axis=1)
        ytest = ytest.argmax(axis=1)

        ACC = accuracy_score(ytest, y_pred)
        print('\nClassification Accuracy (ACC): {0:0.1f}'.format(ACC*100) + '%')

        print("\nConfusion Matrix")                                          
        cm = confusion_matrix(ytest, y_pred)

        sns.heatmap(cm, cbar=False, annot=True,fmt="d", linewidths=.5,
              yticklabels=['A','B','C','D','E','F','G','H','I'],
              xticklabels=['A','B','C','D','E','F','G','H','I'])
        plt.xlabel('True label')
        plt.ylabel('Predicted label')
        plt.show()

        print(classification_report(y_pred, ytest))

    def TestEC(self, test_data, test_labels, threshold):
        if('A' in test_labels):
           test_labels = charToInt(test_labels);

        predictions = self.model.predict(test_data)
        for index, pred in enumerate(predictions):
          if(max(pred) < threshold):
            print(max(pred))
            print("b4",predictions[index],index)
            predictions[index] = np.zeros((len(pred)))
            predictions[index][0] = 1; # 1 in the zero position
            print(predictions[index],index)

        y_pred = np.argmax(predictions, axis=1)

        y_pred = np.where(y_pred == 0, -1, y_pred)
    
        # print("[2]",list(y_pred).count(-1))
        # print("[3]",list(test_labels).count(-1))

        ACC = accuracy_score(test_labels, y_pred)
        print('\nClassification Accuracy (ACC): {0:0.1f}'.format(ACC*100) + '%')

        print("\nConfusion Matrix")                                          
        cm = confusion_matrix(test_labels, y_pred)

        sns.heatmap(cm, cbar=False, annot=True,fmt="d", linewidths=.5,
              yticklabels=['unknown','A','B','C','D','E','F','G','H','I'],
              xticklabels=['unknown','A','B','C','D','E','F','G','H','I'])
        plt.xlabel('True label')
        plt.ylabel('Predicted label')
        plt.show()

        print(classification_report(y_pred, test_labels))


    

    def SaveModel(self, filename):
        self.model.save(filename)
    
    def LoadModel(self, filename):
        self.model = load_model(filename)

    def Plot(self):
        plot_model(self.model)


In [ ]:
def charToInt(data):
    charToInt = {'A': 1, 'B': 2,'C': 3, 'D': 4,'E': 5,'F': 6,'G' : 7, 'H': 8, 'I': 9, 'unknown': -1}
    temp = np.zeros(len(data))
    for index, element in enumerate(data):
        temp[index] = charToInt[element[0]]
    return temp

In [ ]:
from sklearn.model_selection import train_test_split

data = np.load('data.npy')
labels = np.load('labels.npy')

Xtrain, Xtest, ytrain, ytest = train_test_split(data, labels, random_state=42, test_size=0.1)

Xtrain = Xtrain.astype(np.float32) / 255.0
Xtest = Xtest.astype(np.float32) / 255.0

ytestChars = ytest
ytrain = keras.utils.to_categorical(charToInt(ytrain))
ytest = keras.utils.to_categorical(charToInt(ytest))

print(Xtrain[0].dtype)
Xtrain.shape, ytrain.shape, Xtest.shape, ytest.shape

In [ ]:
numClasses = 10
input_shape =(100,100,3)
batch_size = 64
epochs=130
save_model_filepath = 'test_model.h5'

#Train the classifier
model = Classifier(numClasses=numClasses, input_shape=input_shape, batch_size=batch_size, epochs=epochs)
model.Plot()

model.Train(Xtrain, ytrain, Xtest, ytest, save_model_filepath)